In [1]:
from copy import deepcopy
"""
Class: Literals
"""

class Literal:
    def __init__(self,name='',sign=True):
        self.name = name
        self.sign = sign
        
    def __str__(self):
        if self.sign:
            return self.name
        elif self.name == 'False':
            return self.name
        else:
            return "-" + self.name
    
    def __neg__(self):
        negLiteral = Literal(self.name)
        negLiteral.sign = False
        return negLiteral
    
    def __eq__(self, other):
        # Equality is independent of sign
        return self.name == other.name
    
    def __hash__(self):
        # Hash is independent of sign
        return hash(self.name)
    
    def __getitem__(self, item):
        return self

In [2]:
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')
T = Literal('True')
Fal = Literal('False',False)

KB = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, 
      {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]
KB1 = [{-A, B, E}, {-B, A}, {-E, A}, {T}, 
      {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}] #D is true
KB2 = [{B, E}, {T}, {T}, {T}, 
      {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}] #A is true
KB3 = [{T}, {T}, {T}, {T}, 
      {-C, -F}, {T}, {F}, {C}] #B is true

symbols = [A,B,C,D,E,F]
symbols1 = [B,C,E,F]

In [3]:
def get_pure_symbol(clauses,symbols):
    
    #checking whether each symbol is pure
    def check_pure(symbol):
        pure_symbol= None
        #print "hello world\n\n\n\n\n\n"
        #print symbol
        for clause in clauses:
            #print clause
            #if the symbol searching is in the clause
            if symbol in clause:
                #print "hello"
                #look through the literals in the clause
                for literal in clause:
                    #if the literal is the symbol
                    if literal == symbol:
                        #if there is no pure symbol in memory
                        if pure_symbol is None:
                            pure_symbol = literal
                        else:
                            if literal.sign != pure_symbol.sign:
                                return None, None
                        #print "Pure Symbol: {}".format(pure_symbol)
        if pure_symbol is not None:
            return pure_symbol, pure_symbol.sign
        else:
            return None, None
    
    #looping through all the symbols
    for symbol in symbols:
        (pure_symbol, pure_sign) = check_pure(symbol)
        #if there is a pure symbol already, return the pure symbol
        if pure_symbol is not None:
            return symbol, pure_sign
    
    return None, None

In [4]:
#get_pure_symbol(KB1,symbols1)

In [5]:
"""
get_unit_clause
return: literal name and sign
"""
def get_unit_clause(clauses):
    unit_clause = None
    unit_sign = None
    for clause in clauses:
        if len(clause) == 1 and T not in clause and Fal not in clause:
            for literal in clause:
                return literal, literal.sign
        
    return None, None

In [6]:
#get_unit_clause(KB1)

In [7]:
def Simplify(clauses,symbol,sign):
    simplifiedClauses = []
    
    def simplify_clause(clause):
        newClause = set()
        if symbol in clause:
            #print "D is in the clause"
            for literal in clause:
                if literal == symbol:
                    if literal.sign == True and sign == True:
                        newClause.clear()
                        newClause.add(T)
                        break
                    elif literal.sign == False and sign == False:
                        newClause.clear()
                        newClause.add(T)
                        break
                    else:
                        continue
                else:
                    newClause.add(literal)
            if len(newClause) == 0:
                newClause.add(Fal)
            return newClause
        else:
            newClause = clause
            return newClause
    
    for clause in clauses:
        sim_clause = simplify_clause(clause)
        #print "This is the sim_clause: {}".format(sim_clause)
        simplifiedClauses.append(sim_clause)
    if len(simplifiedClauses) > 0:
        print "Here are the simplified clauses:"    
    return simplifiedClauses        

In [8]:
"""
KB = [{T}, {T}, {T}, {T}, 
      {-C}, {T}, {T}, {C}]

simplify = Simplify(KB,-C,False)
for clause in simplify:
    for literal in clause:
        print literal
    print "\n"
"""

'\nKB = [{T}, {T}, {T}, {T}, \n      {-C}, {T}, {T}, {C}]\n\nsimplify = Simplify(KB,-C,False)\nfor clause in simplify:\n    for literal in clause:\n        print literal\n    print "\n"\n'

In [9]:
#print Simplify(KB1,A,True)

In [10]:
"""
DPLLSatisfiable
Arg:
    symbol: (list) sorted list of symbols needed to examine
"""

def DPLLSatisfiable(KB):
    #sat_model: set
    global sat_model
    sat_model = {}
    
    def merge_two_dicts(x, y):
        z = x.copy()   # start with x's keys and values
        z.update(y)    # modifies z with y's keys and values & returns None
        return z
    
    def DPLL(clauses,symbols,model, unit_prop_symbol=None, unit_prop_sign=None):
        if unit_prop_symbol is not None:
            print "\n\n\nUnit Propagating..."
            clauses = Simplify(clauses,unit_prop_symbol,unit_prop_sign)
            #print "Simplified Clauses: {}".format(clauses)
            
        print "==================================="
        print "These are the clauses: {}\n".format(clauses)
        print "These are the symbols: {} ({})\n".format(symbols,len(symbols))
        print "This is the current model: {}\n".format(model)
        print "This is the unit propagating: {} {}\n".format(unit_prop_symbol,unit_prop_sign)
        
        if all([T in clause for clause in clauses]):
            sat_model = model
            print sat_model
            return sat_model, True
        
        """
        if any([Fal in clause for clause in clauses]):
            return model, False
        """
                                                        
        if len(symbols) != 0:
            print "Into the heuristics..."
            pure_symbol, pure_sign = get_pure_symbol(clauses,symbols)
            if pure_symbol is not None:
                print "Pure symbol found: {} {}".format(pure_symbol,pure_sign)
                symbols.remove(pure_symbol)
                print "Number of symbols after pure: {}".format(len(symbols))
                model.update({pure_symbol: pure_sign})
                print "The new model after pure: {}".format(model)
                print "\n===================================\n"
                clause_after_pure = deepcopy(clauses)
                symbols_after_pure = deepcopy(symbols)
                return DPLL(clause_after_pure,symbols_after_pure,model,pure_symbol,pure_sign)

            unit_symbol, unit_sign = get_unit_clause(clauses)
            if unit_symbol is not None:
                print "Unit clause found: {}".format(unit_symbol)
                print "Symbols: {}".format(symbols)
                unit_prop_symbol = unit_symbol
                unit_prop_sign = unit_sign
                symbols.remove(unit_symbol)
                print "Number of symbols after unit: {}".format(len(symbols))
                model.update({unit_symbol: unit_sign})
                print "The new model after unit: {}".format(model)
                print "\n===================================\n"
                clause_after_unit = deepcopy(clauses)
                symbols_after_unit = deepcopy(symbols)
                return DPLL(clause_after_unit,symbols_after_unit,model,unit_symbol,unit_sign)
            
            print "Here are the symbols available to pop: {}".format(symbols)
            p = symbols.pop(0)
            print "Popping this symbol: {}".format(p)
            print "\n===================================\n"
            p_true = {p:True}
            p_false = {p:False}
            p_true = merge_two_dicts(p_true,model)
            p_false = merge_two_dicts(p_false,model)
            clause_after_pop = deepcopy(clauses)
            symbols_after_pop = deepcopy(symbols)
            return DPLL(clause_after_pop,symbols_after_pop,p_true,p,True) or DPLL(clause_after_pop,symbols_after_pop,p_false,p,False)
        
    (sat_model, satisfy) = DPLL(KB,symbols,{}) 
    print "This is the DPLLSatisfy result: {}".format(satisfy)
    if satisfy:
        for symbol in symbols:
            if symbol in sat_model:
                print symbol + ": " + str(sat_model[symbol])
            else:
                print str(symbol) + ": free"

In [11]:
DPLLSatisfiable(KB)

These are the clauses: [set([<__main__.Literal instance at 0x10ddcac20>, <__main__.Literal instance at 0x10ddcae18>, <__main__.Literal instance at 0x10ddcac68>]), set([<__main__.Literal instance at 0x10ddcab90>, <__main__.Literal instance at 0x10de64290>]), set([<__main__.Literal instance at 0x10ddcab90>, <__main__.Literal instance at 0x10de641b8>]), set([<__main__.Literal instance at 0x10de645a8>, <__main__.Literal instance at 0x10ddca368>]), set([<__main__.Literal instance at 0x10de64560>, <__main__.Literal instance at 0x10de7a440>, <__main__.Literal instance at 0x10de7a1b8>]), set([<__main__.Literal instance at 0x10ddcae18>, <__main__.Literal instance at 0x10de7a5f0>]), set([<__main__.Literal instance at 0x10de7a320>, <__main__.Literal instance at 0x10ddcabd8>]), set([<__main__.Literal instance at 0x10ddcaef0>, <__main__.Literal instance at 0x10de7a0e0>])]

These are the symbols: [<__main__.Literal instance at 0x10ddcab90>, <__main__.Literal instance at 0x10ddcae18>, <__main__.Liter

ValueError: list.remove(x): x not in list